# Kristins Alex 

## Imports

In [1]:
import os
from datetime import datetime
from dataclasses import dataclass

import talos as ta
from talos.model import lr_normalizer, early_stopper, hidden_layers

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.initializers import glorot_uniform

available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from numpy.random import seed
seed(1)
tf.random.set_seed(1)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Using TensorFlow backend.


The installed version of TensorFlow 2.1.0 includes GPU support.

Num GPUs Available:  2 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3528456780656821298
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9105744200
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10854106072254051267
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 9104897474
locality {
  bus_id: 1
  links {
  }
}
incarnation: 907602144215201375
physical_device_desc: "device: 1, name: GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5"
]
2 Physical GPUs, 2 Logical GPUs


## AlexNet

In [ ]:
def alexnet(x_train, y_train, x_val, y_val, params):
    
    K.clear_session()
    
    train_generator, valid_generator = create_data_pipline(params['batch_size'], params['samples'])
    tg_steps_per_epoch = train_generator.n // train_generator.batch_size
    vg_validation_steps = valid_generator.n // valid_generator.batch_size
    print('Steps per Epoch: {}, Validation Steps: {}'.format(tg_steps_per_epoch, vg_validation_steps))
    
    
    if params['activation'] == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = params['leaky_alpha'])
    elif params['activation'] == 'relu':
        activation_layer = ReLU()
    
    model = Sequential([
        Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,Global.num_image_channels)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        Dense(4096, activation=activation_layer),
        Dropout(params['dropout']),
        Dense(4096, activation=activation_layer),
        Dropout(params['dropout']),
        Dense(units = 2, activation=activation_layer)
        #Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer = params['optimizer'](lr = lr_normalizer(params['lr'], params['optimizer'])), 
        loss = Global.loss_function, 
        metrics = get_reduction_metric(Global.reduction_metric)
    )
    
    print('Model was compiled')
    print(model.summary())
    print('_________________________________________________________________')
    
    checkpointer = callbacks.ModelCheckpoint(
        filepath = Global.target_dir + 'CNN_ALEX_Model_and_Weights_{}.hdf5'.format(Global.image_channels),
        monitor =  Global.monitor_value,
        verbose = 1,
        save_weights_only = False,
        save_best_only = True,
        mode = 'min'
    )
    print('Checkpointer was created')
    
    csv_logger = callbacks.CSVLogger(
        filename = Global.target_dir + 'CNN_ALEX_Logger_{}.csv'.format(Global.image_channels),
        separator = ',',
        append = False
    )
    print('CSV Logger was created')

    lr_reducer = callbacks.ReduceLROnPlateau(
        monitor = 'val_loss',
        factor = 0.1,
        patience = 13,
        verbose = 1,
        mode = 'min',
        min_delta = 0.0001
    )
    print('Learning Rate Reducer was created')
    
    early_stopper = callbacks.EarlyStopping(
        monitor = 'val_loss',
        min_delta = 0,
        #patience = 15,
        patience = 20,
        verbose = 1,
        mode = 'min',
        restore_best_weights = True
    )
    print('Early Stopper was created')
    
    startTime = datetime.now()
        print(startTime)
    
    out = model.fit(
        x = train_generator,
        epochs = params['epochs'],
        validation_data = valid_generator,
        steps_per_epoch = tg_steps_per_epoch,
        validation_steps = vg_validation_steps,
        callbacks = [checkpointer, csv_logger, lr_reducer, early_stopper],
        workers = 8
    )
    print("Time taken:", datetime.now() - startTime)

    return out, model

### Benutzerdefinierte Kostenfunktion & Metrik

In [ ]:
def mean_absolut_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Hilfsfunktion

In [ ]:
def get_reduction_metric(metric):
    
    if metric == 'mean_absolut_error':
        return [mean_absolut_error]
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

### Automatische Optimizer Generierung aus String

In [ ]:
def make_optimizer(optimizer):
    # [Adam, Nadam, Adagrad, RMSprop]
    if optimizer == "<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>":
        return Adam
    else:
        print('ERROR::: Unspecified Optimizer')

### Struct for global parameter

In [ ]:
@dataclass
class global_parameter:
    loss_function: str = 'mean_squared_error'
    reduction_metric: str = 'mean_absolut_error'
    monitor_value: str = 'val_mean_absolut_error'
        
    net_architecture = 'VGG16' # 'AlexNet'
    
    dataset: str = '201019_2253_final'
    device: str = 'RTX_2080_Ti'
    data_augmentation: bool = True
    image_channels: str = 'rgb' # just change this, everything else will automaticlly adjusted
    num_image_channels: int = 3
    image_dir: str = '..\\..\\data_generation\\dataset\\{}\\'.format(dataset)
    
    csv_file_name: str = 'labels_ks_RGB.csv'
    csv_file: str = image_dir + csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(net_architecture, dataset, image_channels)
    results: str = '\\..\\{}_{}_{}_Results.csv'.format(net_architecture, dataset, image_channels)
    results_man: str = '\\..\\201019_2253_final_rgb_Results_manual.csv'

        
Global = global_parameter

if(Global.image_channels == 'rgba'):
    Global.num_image_channels = 4
    csv_file_name: str = 'lables_ks_RGBD.csv'
    csv_file: str = image_dir + csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(Global.net_architecture, Global.dataset, Global.image_channels)
    results: str = '\\..\\{}_{}_{}_Results.csv'.format(Global.net_architecture, Global.dataset, Global.image_channels)

### Generierung Datenpipeline (Angepasst für Talos)

In [ ]:
def create_data_pipline(batch_size, num_samples):
    
    df = pd.read_csv(Global.csv_file)
    df_shuffled = df.sample(frac = 1, random_state = 1)
    df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]
    df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
    
    if Global.data_augmentation:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.1,
            brightness_range = (0.5, 1.0), 
            fill_mode = 'nearest'
        )
    else:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255
        )
        
    train_generator = train_data_generator.flow_from_dataframe(
        dataframe = df_train,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = True,
        seed = 77,
        batch_size = batch_size
    )
    #print(df_train)
        
    valid_data_generator = ImageDataGenerator(
        rescale = 1./255
    )
    
    valid_generator = valid_data_generator.flow_from_dataframe(
        dataframe = df_valid,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = False,
        seed = 77,
        batch_size = batch_size
    )
    #print(df_train)
    
    return train_generator, valid_generator

In [ ]:
if(not os.path.exists(Global.target_dir)):
    os.makedirs(Global.target_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(Global.target_dir))

device_file = open(Global.target_dir + '{}.txt'.format(Global.device), "a+")

### Top Ergebnisse

In [ ]:
base_results = Global.target_dir + Global.results_man
df = pd.read_csv(base_results).drop(columns = ['round_epochs', 'samples', 'epochs'], axis = 0)
sort_value = 'val_mean_absolut_error'
df = df.sort_values(sort_value, axis = 0, ascending = True, inplace = False, kind = 'quicksort', na_position = 'last')
print('Displaying: {}'.format(base_results))
df.head(10)

### GridSerach

#### Hyper Parameter

In [ ]:
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
#     Adagrad

global_hyper_parameter = {
    'samples': None,
    'epochs': None,
    'batch_size': None,
    'optimizer': None,
    'lr': None,
    'first_neuron': None,
    'dropout': None,
    'activation': None,
    'leaky_alpha': None,
    'hidden_layers': None,
}


In [ ]:
def get_params(top_results_index):
    
    #     Adam = RMSprop + Momentum (lr=0.001)
    #     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
    #     RMSprop = (lr=0.001)
    #     SGD = (lr=0.01)
    #     Adagrad

    hyper_parameter = global_hyper_parameter

    hyper_parameter['samples'] = [100000]
    hyper_parameter['epochs'] = [400]
    hyper_parameter['batch_size'] = [df.iloc[top_results_index]['batch_size']]
    hyper_parameter['optimizer'] = [make_optimizer(df.loc[top_results_index]['optimizer'])]
    hyper_parameter['lr'] = [df.iloc[top_results_index]['lr']]
    hyper_parameter['first_neuron'] = [df.iloc[top_results_index]['first_neuron']]
    hyper_parameter['dropout'] = [df.iloc[top_results_index]['dropout']]
    hyper_parameter['activation'] = [df.iloc[top_results_index]['activation']]
    hyper_parameter['leaky_alpha'] = [0.1] #Default bei LeakyReLU, sonst PReLU
    hyper_parameter['hidden_layers'] = [df.iloc[top_results_index]['hidden_layers']]
    
    return hyper_parameter

### Test to see if RGBD

In [ ]:
from PIL import Image 
#print(Global.csv_file)
df2 = pd.read_csv(Global.csv_file)
print(df2.head(5))
img_nm = df2.iloc[0]['Filename']
#Image(Global.image_dir + img_nm)

img = Image.open(Global.image_dir + img_nm)
img.show()
for i in range(6):
    print(i, end=', ')

for i in range(224):
    for j in range(224):
        colors = img.getpixel((i,j))
        print(colors)

### Start Talos

In [ ]:
dummy_x = np.empty((1, 2, Global.num_image_channels, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):
    top_results_index = 0

    tmp_dir = '..\\TMP_TALOS_{}'.format(Global.device)
    trained_results = Global.target_dir + 'Talos_Results_Fine_Idx{}.csv'.format(Global.image_channels)

    startTime = datetime.now()

    parameters = get_params(top_results_index)

    t = ta.Scan(
        x = dummy_x,
        y = dummy_y,
        model = alexnet,
        params = parameters,
        experiment_name = tmp_dir,
        #shuffle=False,
        reduction_metric = get_reduction_metric(Global.reduction_metric),
        disable_progress_bar = False,
        print_params = True,
        clear_session = True
    )

    print("Time taken:", datetime.now() - startTime)

    print('Writing Device File')
    device_file.write('Trained Model: {}'.format(Global.image_channels))
    device_file.close()
    
    df_experiment_results = pd.read_csv(tmp_dir + '\\' + os.listdir(tmp_dir)[0])
    df_experiment_results['Base'] = None
    for i in range(df_experiment_results.shape[0]):
        df_experiment_results['Base'][i] = Global.image_channles

    if os.path.isfile(trained_results):
        df_experiment_results.to_csv(trained_results, mode = 'a', index = False, header = False)
    else:
        df_experiment_results.to_csv(trained_results, index = False)

    shutil.rmtree(tmp_dir)